# Title

In [ ]:
"""
    TITLE   :
    PROJECT :
""";

__author__ = ''
__version__ = ''

<span style='font-size:30px;font-weight:650'>
    About
</span>

information about this notebook


<br><br>

- - - 

## Prepare


### Imports

In [2]:
# THIRD PARTY
import numpy as np

import astropy.coordinates as coord
import astropy.units as u
from astropy.visualization import quantity_support

from galpy import potential as gpot
from galpy import df as gdf
import agama

# PROJECT-SPECIFIC
from discO import conf
from discO import PotentialSampler, GaussianMeasurementErrorSampler

In [3]:
quantity_support();

### Parameters

In [4]:
mass = 1e12 * u.solMass
r0 = 10 * u.kpc  # scale factor

<br><br>

- - - 

## Code

In [5]:
hernquist_pot = gpot.HernquistPotential(amp=mass, a=r0)

In [6]:
sampler = PotentialSampler(gdf.isotropicHernquistdf(hernquist_pot))
# sampler.sample(n=10)

In [7]:
samples = sampler.resample(niter=2, n=3, frame=None, sample_axis=-1, random=None)
samples

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [[(181.03038185,   8.86964772, 11.72871177),
      (272.50982481,   8.56533101,  7.50222492),
      (115.38231971,  77.38565247, 10.14305171)],
     [(205.83040344, -47.64782902,  8.24700861),
      (  1.33666106, -11.77002859, 25.46644816),
      (228.47298224, -30.5859229 , 48.75079335)]]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [[(-0.7840572 ,  -7.58361473, 149.82775425),
      (-6.78746292, -10.77064775,  85.76884882),
      ( 0.86476185,  -3.67108727,  60.45524089)],
     [( 0.51470008,   7.50900185, 139.60190371),
      ( 0.89328421,  -2.01026422, -87.82292783),
      (-0.71694177,  -0.28500617,  45.41169304)]]>

In [8]:
samples.ra

<Longitude [[181.03038185, 272.50982481, 115.38231971],
            [205.83040344,   1.33666106, 228.47298224]] deg>

In [9]:
samples.ra.mean(axis=-1)

<Longitude [189.64084212, 145.21334891] deg>

AGAMA

In [10]:
pot_Hern = agama.Potential(type='Spheroid', mass=1e12, scaleRadius=10, gamma=1, alpha =1, beta=4, cutoffStrength=0)

In [11]:
sampler = PotentialSampler(
    agama.Potential(type='Spheroid', mass=1e12, scaleRadius=10, gamma=1, alpha =1, beta=4, cutoffStrength=0),
)

sampler

In [12]:
samples = sampler.sample(n=10)
samples

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(269.77768587, -80.39242629,  12.15818053),
     (211.53383512,  -3.67881258,   7.37721302),
     (135.4959591 , -44.62636438, 156.25820005),
     (  3.85684546,  -7.46999137,   5.08874191),
     ( 42.11155402, -20.90390085,   7.7856392 ),
     (212.56246467, -64.15957604,  16.58761413),
     (203.11313493,  -9.16456976,   6.31741618),
     ( 61.49015059, -33.66474899,   3.83061213),
     (344.11597718, -41.05292432,   4.97326983),
     ( 98.63435577,  16.56923216,  32.21408322)]>

In [13]:
samples.potential

In [14]:
array = sampler.resample(niter=20, n=3)

array

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [[(129.03524334, -56.9676308 , 9.61517737e+01),
      (168.19072372, -43.2141901 , 2.03577298e+01),
      (143.38269417,  37.60374365, 2.20657715e+01)],
     [(132.33728927,   1.01790775, 1.94559012e+01),
      (217.32849778, -12.92419406, 2.44626360e+01),
      (353.59709381, -25.25701904, 3.40593284e+00)],
     [(158.8787906 , -32.67094052, 2.12963796e+01),
      (245.13396814,  26.6052406 , 1.82077872e+01),
      (189.41093455,  23.48182559, 4.59594112e+00)],
     [( 71.70686227, -18.6030652 , 2.85785231e+01),
      (358.01572722,  57.00733972, 9.89492176e+00),
      (253.62991614,  76.03126032, 3.55818643e+00)],
     [(154.17532143, -53.6948407 , 1.12427032e+01),
      (113.52402778,  50.74812545, 9.94153299e+00),
      ( 50.17935374, -18.51390405, 4.46464834e+01)],
     [(344.62263107,  61.312028  , 4.39212405e+00),
      (168.93917844,  -3.49862856, 2.30554913e+01),
      (290.89025883, -37.26150772, 8.62462443e+00)],
 

---

Now pipe with fitters

In [15]:
sampler = PotentialSampler(gdf.isotropicHernquistdf(hernquist_pot))
fitter = PotentialFitter("multipole", package="agama", return_specific_class=True)
ms = GaussianMeasurementErrorSampler()

pipe = sampler | ms | fitter

fit_pot = pipe(n=10000)

fit_pot

NameError: name 'PotentialFitter' is not defined

In [ ]:
ms(sampler.sample(n=3), c_err=0.01)

In [ ]:
from astropy.utils.misc import NumpyRNGContext
import numpy as np

with NumpyRNGContext(0):
    random = np.random.default_rng(0)
    print(random.uniform(10))

<br><br>

- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>